# 데이터 전처리

시설물 데이터의 경우 노원과 인접구 데이터만 갖도록 가공, 포함범위 : 노원 도봉 중랑 성북

라이브러리 불러오기

In [6]:
import numpy as np
import pandas as pd

상수 지정

In [7]:
gu_list=["노원구", "도봉구", "중랑구", "성북구"]
gu_code=[3100000, 3090000, 3060000, 3070000]
start_path = "전처리필요데이터/"
end_path = "전처리완료데이터/"
encoding = 'cp949'

### 서울시 장소별 범죄발생건수 가공

장소끼리 건수 더한 후, 확률로 변환, 쓰기쉽게 전치

In [3]:
seoul_crime = pd.read_csv(start_path+"seoul_crime.csv", encoding=encoding)
seoul_crime

,장소,발생건수
0,"아파트,연립다세대",30
1,단독주택,26
2,고속도로,0
3,노상,17
4,상점,2
...,...,...
130,해상,0
131,부대,4
132,구금장소,8
133,공지,0


In [4]:
temp = seoul_crime.groupby("장소")
temp = temp.sum()
temp /= temp.sum()
temp.columns=["확률"]
temp = pd.DataFrame(temp.T, columns=temp.index)
temp

장소,고속도로,공사장광산,공장,공지,구금장소,금융기관,기타,기타교통수단내,노상,단독주택,...,"역,대합실",유원지,유흥접객업소,의료기관,종교기관,지하철,창고,학교,해상,흥행장
확률,0.000454,0.003872,0.000841,0.000011,0.000088,0.007168,0.180279,0.014701,0.32434,0.087457,...,0.01187,0.010642,0.065698,0.007179,0.001892,0.016892,0.001117,0.004735,0.0,0.001239


유사한 분류 묶은후, 저장

In [5]:
processed_crime = pd.DataFrame()
processed_crime["공업"] = temp[["공사장광산", "공장", "창고"]].sum(axis=1)
processed_crime["공원"] = temp[["유원지", "공지"]].sum(axis=1)
processed_crime["교통"] = temp[["기타교통수단내", "지하철", "역,대합실"]].sum(axis=1)
processed_crime["기타"] = temp[["구금장소", "고속도로", "기타", "부대"]].sum(axis=1)
processed_crime["사무"] = temp[["금융기관", "사무실"]].sum(axis=1)
processed_crime["숙박"] = temp[["숙박업소목욕탕"]].sum(axis=1)
processed_crime["상업"] = temp[["시장,노점", "상점", "흥행장"]].sum(axis=1)
processed_crime["아파트"] = temp[["아파트,연립다세대"]].sum(axis=1)
processed_crime["야외"] = temp[["노상", "산야"]].sum(axis=1)
processed_crime["유흥"] = temp[["유흥접객업소"]].sum(axis=1)
processed_crime["의료"] = temp[["의료기관"]].sum(axis=1)
processed_crime["종교"] = temp[["종교기관"]].sum(axis=1)
processed_crime["주택"] = temp[["단독주택"]].sum(axis=1)
processed_crime["학교"] = temp[["학교"]].sum(axis=1)
processed_crime

,공업,공원,교통,기타,사무,숙박,상업,아파트,야외,유흥,의료,종교,주택,학교
확률,0.00583,0.010653,0.043463,0.180887,0.025199,0.010841,0.123032,0.108198,0.324937,0.065698,0.007179,0.001892,0.087457,0.004735


In [6]:
processed_crime.to_csv(end_path+"processed_crime.csv", index=False,  encoding=encoding)

### 서울시 cctv가공

In [7]:
seoul_cctv = pd.read_csv(start_path+"seoul_cctv.csv", encoding=encoding)
seoul_cctv

,자치구,위도,경도
0,중랑구,37.5809,127.0848
1,중랑구,37.5809,127.0848
2,중랑구,37.5822,127.0839
3,중랑구,37.5791,127.0859
4,중랑구,37.5820,127.0953
...,...,...,...
83729,강남구,37.5059,127.0589
83730,강남구,37.4902,127.1014
83731,강남구,37.4902,127.1014
83732,강남구,37.5059,127.0589


In [8]:
processed_cctv = pd.DataFrame(columns=["위도", "경도"], dtype=np.float64)
for i in gu_list:
    temp = seoul_cctv.loc[seoul_cctv["자치구"]==i,["위도", "경도"]]
    processed_cctv = pd.concat([processed_cctv, temp])
processed_cctv

,위도,경도
48581,37.6734,127.0590
48582,37.6734,127.0590
48583,37.6734,127.0590
48584,37.6729,127.0588
48585,37.6729,127.0588
...,...,...
26702,37.5985,127.0327
26703,37.5985,127.0327
26704,37.5881,127.0075
26705,37.6096,127.0572


In [9]:
processed_cctv.to_csv(end_path+"processed_cctv.csv", index=False,  encoding=encoding)

### 서울시 안심귀가길 시설물 가공

한 열에 point(경도 위도)형식으로 저장 -> 위경도 추출

In [10]:
seoul_ansim = pd.read_csv(start_path+"seoul_ansim.csv", encoding=encoding)
seoul_ansim

,포인트 wkt,시군구명
0,POINT (126.968590563668 37.5793826677127),서울특별시 종로구
1,POINT (126.968590563668 37.5793826677127),서울특별시 종로구
2,POINT (126.968590563668 37.5793826677127),서울특별시 종로구
3,POINT (126.968590563668 37.5793826677127),서울특별시 종로구
4,POINT (126.968535907548 37.579433000699),서울특별시 종로구
...,...,...
11878,POINT (127.1324431 37.540174),서울특별시 강동구
11879,POINT (127.1323799 37.5401786),서울특별시 강동구
11880,POINT (127.1323799 37.5401786),서울특별시 강동구
11881,POINT (127.1323799 37.5401786),서울특별시 강동구


In [11]:
processed_ansim = pd.DataFrame()
for i in gu_list:
    processed_ansim = pd.concat([ processed_ansim, seoul_ansim.loc[seoul_ansim["시군구명"] == "서울특별시 "+i, "포인트 wkt"] ])

def word_to_EW(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[0]
def word_to_NS(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[1][:-1]

processed_ansim["위도"] = processed_ansim[0].apply(word_to_NS)
processed_ansim["경도"] = processed_ansim[0].apply(word_to_EW)
processed_ansim = processed_ansim.drop(columns=[0]).reset_index(drop=True)
processed_ansim

,위도,경도
0,37.624871,127.050828
1,37.624608,127.050897
2,37.624236,127.051045
3,37.624017,127.051316
4,37.624017,127.051316
...,...,...
2033,37.60735199999999878,127.05262000000000455
2034,37.60735700000000037,127.0526660000000021
2035,37.60735700000000037,127.0526660000000021
2036,37.6074300000000008,127.05291400000000124


In [12]:
processed_ansim.to_csv(end_path+"processed_ansim.csv", index=False, encoding=encoding)

### 서울시 보안등 가공

하려했으나 중랑구, 성북구는 데이터가 없어 제외

In [13]:
nowon_light = pd.read_csv(start_path+"nowon_light.csv", encoding=encoding)
nowon_light

,위도,경도
0,37.667069,127.067414
1,37.666447,127.071109
2,37.658859,127.067973
3,37.616181,127.066786
4,37.661095,127.070707
...,...,...
652486,37.623603,127.063553
652487,37.623819,127.063466
652488,37.623776,127.063446
652489,37.623573,127.063403


In [14]:
dobong_light = pd.read_csv(start_path+"dobong_light.csv", encoding=encoding)
dobong_light

,위도,경도
0,37.659426,127.036036
1,37.660503,127.040867
2,37.658717,127.038386
3,37.653005,127.035297
4,37.656666,127.037624
...,...,...
5913,37.664634,127.028074
5914,37.662302,127.024533
5915,37.664823,127.027847
5916,37.661761,127.023063


In [15]:
processed_light = pd.concat([nowon_light, dobong_light]).reset_index(drop=True)
processed_light

,위도,경도
0,37.667069,127.067414
1,37.666447,127.071109
2,37.658859,127.067973
3,37.616181,127.066786
4,37.661095,127.070707
...,...,...
658404,37.664634,127.028074
658405,37.662302,127.024533
658406,37.664823,127.027847
658407,37.661761,127.023063


In [16]:
processed_light.to_csv(end_path+"processed_light.csv", index=False,  encoding=encoding)

### 서울시 유흥주점 가공

범위내 영업중 장소 골라냄

In [20]:
seoul_salon = pd.read_csv(start_path+"seoul_salon.csv", encoding=encoding)
seoul_salon

,개방자치단체코드,상세영업상태명,지번주소,도로명주소
0,3220000,영업,"서울특별시 강남구 역삼동 678-24 늘봄빌딩 2,3층","서울특별시 강남구 테헤란로27길 8-4, 늘봄빌딩 (2,3층) (역삼동)"
1,3230000,영업,서울특별시 송파구 가락동 98-7,"서울특별시 송파구 송파대로28길 13, 지하1층 101호 (가락동, 거북이빌딩)"
2,3130000,폐업,서울특별시 마포구 합정동 382-22번지 지하1층,NaN
3,3180000,영업,서울특별시 영등포구 신길동 449-22 신성빌딩,"서울특별시 영등포구 영등포로 400, 신성빌딩 지하1층 (신길동)"
4,3210000,영업,서울특별시 서초구 잠원동 12-21,서울특별시 서초구 강남대로 615 (잠원동)
...,...,...,...,...
4932,3180000,영업,"서울특별시 영등포구 여의도동 44-32 에리트빌팅 지하101,102,103호","서울특별시 영등포구 여의대방로65길 12 (여의도동, 에리트빌딩 지하101,102,..."
4933,3240000,폐업,서울특별시 강동구 길동 415-7 2층,"서울특별시 강동구 천호대로175길 55, 2층 (길동)"
4934,3240000,폐업,서울특별시 강동구 길동 415-7,서울특별시 강동구 천호대로175길 55 (길동)
4935,3240000,폐업,서울특별시 강동구 길동 415-7 지층,"서울특별시 강동구 천호대로175길 55 (길동,지층)"


In [21]:
seoul_salon = seoul_salon.loc[seoul_salon["상세영업상태명"]=="영업", :]
temp_salon = pd.DataFrame()
for i in gu_code:
    temp_salon = pd.concat([ temp_salon, seoul_salon.loc[seoul_salon["개방자치단체코드"] == i, ["지번주소", "도로명주소"]] ])
temp_salon = temp_salon.reset_index(drop=True)
temp_salon

,지번주소,도로명주소
0,서울특별시 노원구 상계동 706-1 2층,"서울특별시 노원구 노해로75길 20-21 (상계동,2층)"
1,서울특별시 노원구 상계동 710 문화빌딩 지하1층,"서울특별시 노원구 노해로77길 14-3, 지하1층 (상계동, 문화빌딩)"
2,서울특별시 노원구 상계동 730-5 지하2호,"서울특별시 노원구 동일로 1393, 지하층 지하2호 (상계동)"
3,서울특별시 노원구 상계동 711-9 노블레스관광호텔 3층,"서울특별시 노원구 노해로77길 22, 노블레스관광호텔 3층 (상계동)"
4,서울특별시 노원구 상계동 734-4번지 동방빌딩,"서울특별시 노원구 노해로 456, 동방빌딩 지하층 (상계동)"
...,...,...
60,서울특별시 성북구 동선동4가 7 외 1필지,"서울특별시 성북구 아리랑로 4 (동선동4가,외 1필지)"
61,서울특별시 성북구 동소문동5가 85번지,서울특별시 성북구 동소문로20길 12 (동소문동5가)
62,서울특별시 성북구 동선동1가 2-12번지,서울특별시 성북구 동소문로20나길 3 (동선동1가)
63,서울특별시 성북구 보문동4가 77-2번지 브라운관광호텔 지하1층,"서울특별시 성북구 보문로 78 (보문동4가,브라운관광호텔 지하1층)"


해당데이터는 주소로만 이뤄져있으므로 위경도로 변환해야함

https://www.geoservice.co.kr/ 이용

In [22]:
temp_salon.to_csv(start_path+"temp_salon.csv", index=False,  encoding=encoding)

위경도 변환된 데이터 확인

In [24]:
processed_salon = pd.read_csv(end_path+"processed_salon.csv", encoding=encoding)
processed_salon

,지번주소,도로명주소
0,서울특별시 노원구 상계동 706-1 2층,"서울특별시 노원구 노해로75길 20-21 (상계동,2층)"
1,서울특별시 노원구 상계동 710 문화빌딩 지하1층,"서울특별시 노원구 노해로77길 14-3, 지하1층 (상계동, 문화빌딩)"
2,서울특별시 노원구 상계동 730-5 지하2호,"서울특별시 노원구 동일로 1393, 지하층 지하2호 (상계동)"
3,서울특별시 노원구 상계동 711-9 노블레스관광호텔 3층,"서울특별시 노원구 노해로77길 22, 노블레스관광호텔 3층 (상계동)"
4,서울특별시 노원구 상계동 734-4번지 동방빌딩,"서울특별시 노원구 노해로 456, 동방빌딩 지하층 (상계동)"
...,...,...
60,서울특별시 성북구 동선동4가 7 외 1필지,"서울특별시 성북구 아리랑로 4 (동선동4가,외 1필지)"
61,서울특별시 성북구 동소문동5가 85번지,서울특별시 성북구 동소문로20길 12 (동소문동5가)
62,서울특별시 성북구 동선동1가 2-12번지,서울특별시 성북구 동소문로20나길 3 (동선동1가)
63,서울특별시 성북구 보문동4가 77-2번지 브라운관광호텔 지하1층,"서울특별시 성북구 보문로 78 (보문동4가,브라운관광호텔 지하1층)"
